# Pair Programming ETL I


Nuestros jefes están especialmente interesados en los datos de generación de las energías renovables y no renovables a nivel nacional. 

En concreto, nos piden sacar toda la información para las fechas de 1 de Enero del 2011 hasta el 31 de Diciembre del 2022. De toda la información que tenemos de generación queremos la evolucion de renovable y no renovable. Además deberemos incluir una columna nueva identificando si el dato corresponde a energía renovable o no renovable. 

NOTA: nos piden que el código debe estar en una función. Al final deberemos tener un dataframe como el siguiente:

Sacaremos la misma información que en el ejericio anterior, pero en este caso por CCAA. Para esto necesitaremos los códigos de cada una de las comunidades autónomas que encontraréis en el siguiente diccionario:

In [2]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}


                    
El resultado final de este conjunto de datos debe ser el siguiente:

Guardad los csv ya que mañana los seguiremos usando en los siguientes pairs

In [3]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import pyjsonviewer
import json

from datetime import datetime, timedelta

In [4]:
url = 'https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date=2011-01-01T00:00&end_date=2011-12-31T23:59&time_trunc=day'

response = requests.get(url=url)

response.status_code

response.reason

200

'OK'

In [5]:
response_json = response.json()

response_json

{'data': {'type': 'Evolución de la generación renovable/no renovable',
  'id': 'gen2',
  'attributes': {'title': 'Evolución de la generación renovable/no renovable',
   'last-update': '2019-06-12T15:32:24.000+02:00',
   'description': None},
  'meta': {'cache-control': {'cache': 'HIT',
    'expireAt': '2023-05-25T02:07:34'}}},
 'included': [{'type': 'Renovable',
   'id': '10299',
   'groupId': None,
   'attributes': {'title': 'Renovable',
    'description': 'Energía renovable: incluye hidráulica, hidroeólica, eólica, solar fotovoltaica, solar térmica, otras renovables y residuos renovables.',
    'color': '#92d050',
    'type': 'Renovable',
    'magnitude': None,
    'composite': False,
    'last-update': '2019-06-12T15:32:24.000+02:00',
    'values': [{'value': 182024.982,
      'percentage': 0.29012086963392214,
      'datetime': '2011-01-01T00:00:00.000+01:00'},
     {'value': 265705.8895,
      'percentage': 0.3874178470195953,
      'datetime': '2011-01-02T00:00:00.000+01:00'},
  

In [6]:
response_json.keys()

dict_keys(['data', 'included'])

In [7]:
response_json['included'][0].keys()

dict_keys(['type', 'id', 'groupId', 'attributes'])

In [8]:
response_json['included'][0]['attributes'].keys()

dict_keys(['title', 'description', 'color', 'type', 'magnitude', 'composite', 'last-update', 'values'])

In [9]:
response_json['included'][0]['attributes'].values()

dict_values(['Renovable', 'Energía renovable: incluye hidráulica, hidroeólica, eólica, solar fotovoltaica, solar térmica, otras renovables y residuos renovables.', '#92d050', 'Renovable', None, False, '2019-06-12T15:32:24.000+02:00', [{'value': 182024.982, 'percentage': 0.29012086963392214, 'datetime': '2011-01-01T00:00:00.000+01:00'}, {'value': 265705.8895, 'percentage': 0.3874178470195953, 'datetime': '2011-01-02T00:00:00.000+01:00'}, {'value': 249489.5735, 'percentage': 0.2959834749565491, 'datetime': '2011-01-03T00:00:00.000+01:00'}, {'value': 270981.713, 'percentage': 0.3109371961918505, 'datetime': '2011-01-04T00:00:00.000+01:00'}, {'value': 379833.015, 'percentage': 0.4528243362421104, 'datetime': '2011-01-05T00:00:00.000+01:00'}, {'value': 380223.24, 'percentage': 0.5169967951516399, 'datetime': '2011-01-06T00:00:00.000+01:00'}, {'value': 418203.2135, 'percentage': 0.527792007319851, 'datetime': '2011-01-07T00:00:00.000+01:00'}, {'value': 414962.006, 'percentage': 0.54281327792

In [10]:
df = pd.DataFrame(response_json['included'][0]['attributes']['values'])

In [11]:
df.head()

,value,percentage,datetime
0,182024.9820,0.290121,2011-01-01T00:00:00.000+01:00
1,265705.8895,0.387418,2011-01-02T00:00:00.000+01:00
2,249489.5735,0.295983,2011-01-03T00:00:00.000+01:00
3,270981.7130,0.310937,2011-01-04T00:00:00.000+01:00
4,379833.0150,0.452824,2011-01-05T00:00:00.000+01:00


In [12]:
def energías_renovables(inicio, fin):
    datos_energia = pd.DataFrame()

    for i in range(inicio, fin):
        url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={i}-01-01T00:00&end_date={i}-12-31T23:59&time_trunc=year"
        response = requests.get(url=url)
        datos = response.json()

        for e in range(len(datos["included"])):
            x = pd.DataFrame(datos["included"][e]["attributes"]["values"])
            x["tipo_energia"] = datos["included"][e]["attributes"]["title"]

            datos_energia = pd.concat([datos_energia, x], axis=0, ignore_index = True)

    return datos_energia

In [13]:
df_tipos = energías_renovables(2011, 2023)
df_tipos.head(3)

,value,percentage,datetime,tipo_energia
0,8.666705e+07,0.310241,2011-01-01T00:00:00.000+01:00,Renovable
1,1.926868e+08,0.689759,2011-01-01T00:00:00.000+01:00,No renovable
2,8.533872e+07,0.301424,2012-01-01T00:00:00.000+01:00,Renovable


In [14]:
df_renovables = df_tipos[df_tipos["tipo_energia"] == "Renovable"]
df_no_renovables = df_tipos[df_tipos["tipo_energia"] == "No renovable"]

Comunidades

In [15]:
def comunidades_autonomas(inicio, fin):
    datos_energia = pd.DataFrame()

    for i in range(inicio, fin):
        for k,v in cod_comunidades.items():
            
            url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={i}-01-01T00:00&end_date={i}-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={v}"
            response = requests.get(url=url)
            respose_json = response.json()

            for e in range(len(respose_json["included"])):
                x = pd.DataFrame(respose_json["included"][e]["attributes"]["values"])
                x["tipo_energia"] = respose_json["included"][e]["attributes"]["title"]
                x["comunidad"] = k
                x["cod_comunidad"] = v

                datos_energia = pd.concat([datos_energia, x], axis=0, ignore_index = True)

    return datos_energia

In [16]:
df_comunidades = comunidades_autonomas(2011, 2023)
df_comunidades.head(2)

,value,percentage,datetime,tipo_energia,comunidad,cod_comunidad
0,202972.8930,1.000000,2011-01-01T00:00:00.000+01:00,No renovable,Ceuta,8744
1,3589.0825,0.016703,2011-01-01T00:00:00.000+01:00,Renovable,Melilla,8745


In [17]:
df_tipos.to_csv('datos/tipos_energia.csv')

In [18]:
df_comunidades.to_csv('datos/energia_comunidades.csv')